<!-- 
![title](source/title.png)
a normal html comment -->

![title](https://raw.githubusercontent.com/emdemor/prediction-house-prices-on-Brazil/master/source/title.png)



**Author**: Eduardo M. de Morais, MSc.

**Date**: Jul 17 2020

# Table of Contents
1. [Introduction](#intro)
2. [Methodology](#method)
3. [Exploratory Analysis](#exp)
4. [Building the Model](#model)
5. [Realistic Model](#real-model)
6. [Conclusion](#conc)

<a id="intro"></a>
# 1.  Introduction


Through this notebook, I have attempted to predict rental prices from a dataset containing data from Brazil. Of course, this is and study case and some idealized procedure was assumed. The main objective here is to study and understand the data to use the knowledge to construct a model to predict the total rental price. The source dataset has the following columns:


* **city** (*categorical*): City where the house or apartment is located. Here, we have two option of cities that the collector didn't specified. 
* **area** (*integer*): Area of constructed  property (The unity is not informed. I'm am assuming that is squared meters, the most usual in Brazil)
* **rooms** (*integer*): The number of rooms in the property
* **bathroom** (*integer*): The number of bathrooms in the property
* **parking spaces** (*integer*): The number of parking spaces at the garage
* **floor**: (*integer*): The number of the floor (if the property is an apartment)
* **animal** (*boolean*): If is allowed to have animals at the house
* **furniture**: (*boolean*): If is a furnished property
* **hoa** (*float*): Homeowners association fee.
* **rent amount** (*float*): Base value of rent
* **property tax** (*float*): State property tax (ad valorem tax)
* **fire insurance** (*float*): The value of fire insurance
* **total** (*float*): The total money spent on rental activity

<a id="method"></a>
# 2. Methodology
In general, when planning a place to live, we have to take into account all expenses and taxes to choose the best place. Because of this, the adopted target to this dataset will be the **total** column.


In real cases applications, the user has no information concerned to property cost, like  **hoa**, **rent amount**, **property tax**, **fire insurance** and **total**. In general, this variables are mutually correlated, firstly, because **total** is the sum the others and; secondly, because in real life, the taxes and fees are  proportional to property value and, somehow, to the rental price. An applicable model must be capable to predict the rental price based only on **city**, **area**, **rooms**, **bathroom**, **parking spaces**, **floor**, **animal** and **furniture**.   However, since this is a idealized study of case, which objective is only to practice the regression models, let's assume that the cost variables is accessible to user. So, our task here is to define the best model to the **total** rental price from the information of other columns.

To construct a system capable to make good recommendations, it is necessary to understand how the features influence the desired variable. To make this, I will split my work in three stages:

**1. Exploring**: An exploratory analysis, to understand the structure of data, missing values, correlations and etc, to select the best set of features to be used at the analysis.


**2. Modeling**: In this step, I try to find the best and simplest model for describing the data. I do this by testing the influence of features on total values. Since that the definition of a best model is something complicated and subjective, I will choose the model  which best efficiency in the follow approach:

>A. Split dataset into test (33%) and training (67%)

>B. Use training dataset to fit the parameters of model

>C. Use the trained model to predict the **total** of test dataset

>D. Use predicted an original **total** to evaluate the *mean average error* and *R2 score* metrics

The model with the lowest mean average error value will be considered the best model. In case of a tie, the one with the highest R2 will be chosen.

**3. Optimizing**: With the chosen model, I want to give to user a system that returns the distribution of prices from a specified set of features. (*To be implemented in next versions*)

<a id="exp"></a>
# 3. Exploratory Analysis

### Table of Contents
1. [Initializing](#init)
2. [Useful Functions](#use-func)
3. [Dataset](#dataset)
4. [Pre-Processing](#pre-pro) 
5. [Correlation Analysis](#corr-an) 
6. [One Feature Linear Regression](#lin-reg) 
7. [Double Feature Linear Regression](#dbl-reg) 

<a id= "init"></a>
## 3.1 Initializing

In [ ]:
# Pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# Matplotlib and seaborn for visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Scipy for statistics
from scipy import stats

# os to manipulate files
import os

from sklearn.metrics import mean_absolute_error,r2_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures

colors = [ "#3498db", "#e74c3c", "#2ecc71","#9b59b6", "#34495e", "#95a5a6"]

<a id="use-func"></a>
## 3.2 Defining Useful Functions

In [ ]:
def convert_to_number(col,convert_type=int,changes = ['-']):
    
    # string will be considered as object
    if col.dtype.name == 'object':
        col_temp = col.copy()
        
        # Change any occurence in changes to ''
        for change in changes:
                col_temp = col_temp.str.replace(change,'')
                
        # Changes empty string elements for NaN
        col_temp.loc[(col_temp == '')] = np.nan
        
        # Convert to number the not nan elements
        col_temp[col_temp.notna()] = col_temp[col_temp.notna()].astype(convert_type)
        
        # Fill nan elements with the mean
        col_temp = col_temp.fillna(int(col_temp.mean()))
        
        return col_temp
    else:
        return col

In [ ]:
def plot_predictions(X_new,y_new,descr = '',cols = ['area', 'hoa','rent amount','property tax']):
    y_col = 'total'

    #cols = ['area', 'hoa','rent amount','property tax','fire insurance']
    
    k = 0
    for x_col in cols:
        plt.close()
        plt.figure(figsize=(8, 5))
        plt.scatter(X_trn[x_col],y_trn,c='lightgray',label = 'Training Dataset',marker='o',zorder=1)
        plt.scatter(X_new[x_col],y_new, label = 'Predictions on Test Dataset',marker='.', c=colors[k], lw = 0.5,zorder=2,alpha = 0.8)
        #plt.scatter(X_tst[x_col],y_pr_tst, label = 'Predictions',marker='.', c='tab:blue', lw = 0.5,zorder=2)


        plt.xlabel(x_col, size = 18)
        plt.ylabel(y_col, size = 18); 
        plt.legend(prop={'size': 12});
        plt.title(descr+y_col+' vs '+x_col, size = 20);
        #plt.savefig('results/'+descr+y_col+' vs '+x_col+'.png')
        plt.show()
        k += 1

<a id="dataset"></a>
## 3.3 The Dataset

First of all, lets import our dataset and make some initial exploratory data analysis

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Import Dataset
df1 = pd.read_csv(os.path.join(dirname,'houses_to_rent.csv')).drop('Unnamed: 0',axis=1)
#df2 = pd.read_csv(os.path.join(dirname,'houses_to_rent_v2.csv'))
#df2.columns = df1.columns

In [ ]:
df1.info()

<a id="pre-pro"></a>
## 3.4 Pre-Processing

From the ```df1.info()``` method calling, we can see some problems. The **hoa**, **rent amount**, **property tax**, **fire insurance** and **total** columns, that are *float* columns are being interpreted by Pandas as string. That's because the monetary units was placed together the value. Besides this, some ```NaN``` values was indicated by the data collector with string 'Sem info' and 'Incluso'. We need to make a specified pre-processing here. For the *float* columns, I implemented a function called `convert_to_number()` in the [function section](#use-func).

My pre-processing method is:

 * Remove all occurences of 'R\\$' , -,'Sem info' and 'Incluso' from data

 * In the cases where the field is 'Sem info' or 'Incluso', replace by `numpy.nan`
 * In *float* columns, replace `NaN` values for the column `mean`
 * Create dummy variables for categorical features, dropping the last columns to prevent a *dummy variable trap*.
 * Removing outliers. ( I'm assuming outliers as the observations with *Z-score* greater than 4 )

In [ ]:
# elements to remove from the dataset
remove = ['R','$',',','-','Sem info','Incluso']

# columns of numerical data
cols = ['hoa','rent amount','property tax','fire insurance','total','floor']

# Making the substitutions
for col in cols:
    df1[col]  = convert_to_number(df1[col],changes=remove)
    
# converting floor to int 
df1['floor'] = df1['floor'].astype('int')

# Getting dummies
df1[['animal','furniture']] = pd.get_dummies(df1[['animal','furniture']], prefix_sep='_', drop_first=True)

# dealing with outliers
cols = ['area','hoa','property tax','rent amount']
for col in cols:
    df1 = df1[np.abs(stats.zscore(df1[col])) < 4]

Let's see how the dataset is now:

In [ ]:
df1.head(10)

<a id="corr-an"></a>
## 3.5 Correlation Analysis
Now, we want to understand the correlations between features of dataset. Let's plot get the 

In [ ]:
correlations = df1.corr()['total'].abs().sort_values(ascending=False).drop('total',axis=0).to_frame()
correlations.plot(kind='bar');

In [ ]:
#totalprice correlation matrix
k = 10 #number of variables for heatmap
plt.figure(figsize=(16,8))
corr = df1.corr()

hm = sns.heatmap(corr, 
                 cbar=True, 
                 annot=True, 
                 square=True, fmt='.2f', 
                 annot_kws={'size': 10}, 
                 yticklabels=corr.columns.values,
                 xticklabels=corr.columns.values,
                 cmap="YlGnBu")
plt.show()

We can see here that there is strong correlation between **rent amount**, **fire insurance** and **total**. As the **total** column is the sum of **hoa**, **rent amount**, **property tax** and **fire insurance**, its natural the appearing of this kind of correlations. This is a problem because, if we use some of this as a target, we have at least 1 spurious degree of freedom in our statistical model. So, as my approach is to predict the **total**, we have to drop or **fire insurance** or **rent amount**. Here, I'll choose to take off **fire insurance** since **rent amount** is a quantity of greater importance.

The first approach that we will take here is the linear fit of the two most correlated features with 'total' (except from 'fire insurance') to see, from the error analysis, how much they contribute to 'total'



<a id="lin-reg"></a>

## 3.6 One Feature Linear Regression

With this section, i want to quantify the predictability of an one feature regression. First of all, lets consider only **rent amount**, that is the most correlated feature.

In [ ]:
# Selecting features and target
x_col = 'rent amount'
y_col = 'total'

X = df1[[x_col]]
y = df1[y_col]

# splitting
X_trn, X_tst, y_trn, y_tst = train_test_split(X, y, test_size=0.333)

# Create regression object
MLR = linear_model.LinearRegression()

poly = PolynomialFeatures(degree=1)
X_trn_pl = poly.fit_transform(X_trn)
X_tst_pl = poly.fit_transform(X_tst)


# Train the model using the training sets
MLR.fit(X_trn_pl,y_trn)

y_pr_tst = MLR.predict(X_tst_pl)
y_pr_trn = MLR.predict(X_trn_pl)

mae = mean_absolute_error(y_tst,y_pr_tst)
r2 = r2_score(y_tst,y_pr_tst)

print('MAE:{:7.2f},{:7.2f}% of mean'.format(mae,100*mae/y_pr_tst.mean()))
print('R2:{:6.3f}'.format(r2))

# Plotting
plt.figure(figsize=(8, 5))
plt.scatter(X_tst,y_tst,c='lightgray',label = 'observations',alpha = 0.6,marker='.',zorder=1)
plt.plot(X_tst,y_pr_tst, label = 'Predictions', c='tab:blue', lw = 3,zorder=2)
plt.xlabel(x_col, size = 18)
plt.ylabel(y_col, size = 18); 
plt.legend(prop={'size': 16});
plt.title(y_col+' vs '+x_col, size = 20);

We can see here that the R2 score is relatively good, but the error is to big: something between 14% and 15% of mean. That's not a bad thing because, despite the strong correlation between them, we have residual freedom to model the other features.

<a id="dbl-reg"></a>
## 3.7 Double Feature Linear Regression

The two features most correlated  with **total** is **rent amount** and **fire insurance**. So, adopting an extremely innocent way of thinking, if we make a 2-features regression with this, we could argue that this will be the most reasonable 2-features model. That's is an error! When we think like this, we are not considering the effect of correlation.

 To test this, lets get the metrics of this regression

#### Double-Feature Linear Regression - rent amount x fire insurance

In [ ]:
# Selecting features and target
X = df1[['rent amount','fire insurance']]
y = df1['total']

# splitting
X_trn, X_tst, y_trn, y_tst = train_test_split(X, y, test_size=0.333)

# Create regression object
MLR = linear_model.LinearRegression()

poly = PolynomialFeatures(degree=1)
X_trn_pl = poly.fit_transform(X_trn)
X_tst_pl = poly.fit_transform(X_tst)

# Train the model using the training sets
MLR.fit(X_trn_pl,y_trn)

y_pr_tst = MLR.predict(X_tst_pl)
y_pr_trn = MLR.predict(X_trn_pl)

mae = mean_absolute_error(y_tst,y_pr_tst)
r2 = r2_score(y_tst,y_pr_tst)


print('MAE:{:7.2f},{:7.2f}% of mean'.format(mae,100*mae/y_pr_tst.mean()))
print('R2:{:6.3f}'.format(r2))

Now, instead of **fire insurance**, lets get the third most correlated feature with **total**:

In [ ]:
# Selecting features and target
X = df1[['rent amount','bathroom']]
y = df1['total']

# splitting
X_trn, X_tst, y_trn, y_tst = train_test_split(X, y, test_size=0.333)

# Create regression object
MLR = linear_model.LinearRegression()

poly = PolynomialFeatures(degree=1)
X_trn_pl = poly.fit_transform(X_trn)
X_tst_pl = poly.fit_transform(X_tst)

# Train the model using the training sets
MLR.fit(X_trn_pl,y_trn)

y_pr_tst = MLR.predict(X_tst_pl)
y_pr_trn = MLR.predict(X_trn_pl)

mae = mean_absolute_error(y_tst,y_pr_tst)
r2 = r2_score(y_tst,y_pr_tst)


print('MAE:{:7.2f},{:7.2f}% of mean'.format(mae,100*mae/y_pr_tst.mean()))
print('R2:{:6.3f}'.format(r2))

So using **rent amount** and **fire insurance** is statistically equivalent to using  **rent amount** and **bathroom**.
Just for fun, let's get the fourth most correlated feature, ie. **property tax**, to see what happens.

In [ ]:
# Selecting features and target
X = df1[['rent amount','property tax']]
y = df1['total']

# splitting
X_trn, X_tst, y_trn, y_tst = train_test_split(X, y, test_size=0.333)

# Create regression object
MLR = linear_model.LinearRegression()

poly = PolynomialFeatures(degree=1)
X_trn_pl = poly.fit_transform(X_trn)
X_tst_pl = poly.fit_transform(X_tst)

# Train the model using the training sets
MLR.fit(X_trn_pl,y_trn)

y_pr_tst = MLR.predict(X_tst_pl)
y_pr_trn = MLR.predict(X_trn_pl)

mae = mean_absolute_error(y_tst,y_pr_tst)
r2 = r2_score(y_tst,y_pr_tst)


print('MAE:{:7.2f},{:7.2f}% of mean'.format(mae,100*mae/y_pr_tst.mean()))
print('R2:{:6.3f}'.format(r2))

Looks how magical statistics is! Its better two use the first and fourth most correlated feature than first/fourth and first/third. The reason of this is because **fire insurance** and **bathroom** is more correlated with **rent amount** than **property tax** .

With this way of think, we can predict the **rent amount** and **hoa** will be a good predictor pair to **total** since both has significantly correlation with **total** but not with each other.

In [ ]:
# Selecting features and target
X = df1[['rent amount','hoa']]
y = df1['total']

# splitting
X_trn, X_tst, y_trn, y_tst = train_test_split(X, y, test_size=0.333)

# Create regression object
MLR = linear_model.LinearRegression()

poly = PolynomialFeatures(degree=1)
X_trn_pl = poly.fit_transform(X_trn)
X_tst_pl = poly.fit_transform(X_tst)

# Train the model using the training sets
MLR.fit(X_trn_pl,y_trn)

y_pr_tst = MLR.predict(X_tst_pl)
y_pr_trn = MLR.predict(X_trn_pl)

mae = mean_absolute_error(y_tst,y_pr_tst)
r2 = r2_score(y_tst,y_pr_tst)


print('MAE:{:7.2f},{:7.2f}% of mean'.format(mae,100*mae/y_pr_tst.mean()))
print('R2:{:6.3f}'.format(r2))

We can improve this a little bit if we using quadratic regression

In [ ]:
# Selecting features and target
X = df1[['rent amount','hoa']]
y = df1['total']

# splitting
X_trn, X_tst, y_trn, y_tst = train_test_split(X, y, test_size=0.333)

# Create regression object
MLR = linear_model.LinearRegression()

poly = PolynomialFeatures(degree=2)
X_trn_pl = poly.fit_transform(X_trn)
X_tst_pl = poly.fit_transform(X_tst)

# Train the model using the training sets
MLR.fit(X_trn_pl,y_trn)

y_pr_tst = MLR.predict(X_tst_pl)
y_pr_trn = MLR.predict(X_trn_pl)

mae = mean_absolute_error(y_tst,y_pr_tst)
r2 = r2_score(y_tst,y_pr_tst)


print('MAE:{:7.2f},{:7.2f}% of mean'.format(mae,100*mae/y_pr_tst.mean()))
print('R2:{:6.3f}'.format(r2))

<a id="model"></a>
# 4. Building the Model

After encoding the categorical data, we have now a dataset with numerical features and target. The best way to model data like this is with a kind of regression model. Here, let's try:

1. [Linear Regression](#m-lin-reg)

2. [Decision Tree](#m-dec-tree)

3. [Random Forest](#m-rnd-frst)

4. [Results](#m-res)

Firstly, let's split the dataset into training and test

In [ ]:
# Selecting features and target
#X = df1.drop('total',axis=1).copy()
X = df1.drop(['total','fire insurance'],axis=1).copy()
y = df1['total'].copy()

# splitting
X_trn, X_tst, y_trn, y_tst = train_test_split(X, y, test_size=0.333)

# dictionary
model_mae = {}
model_r2 = {}

<a id="m-lin-reg"></a>
## 4.1. Linear Regression

In [ ]:
# Create regression object
MLR = linear_model.LinearRegression()

poly = PolynomialFeatures(degree=1)
X_trn_pl = poly.fit_transform(X_trn)
X_tst_pl = poly.fit_transform(X_tst)
MLR.fit(X_trn_pl,y_trn)

y_pr_tst = MLR.predict(X_tst_pl)
mae = mean_absolute_error(y_tst,y_pr_tst)
r2 = r2_score(y_tst,y_pr_tst)

model_mae['linear'] = mae
model_r2['linear'] = r2

print('MAE:{:7.2f},{:7.2f}% of mean'.format(mae,100*mae/y_pr_tst.mean()))
print('R2:{:6.3f}'.format(r2))

In [ ]:
# Getting a reduced sample to test
size = len(X_tst)
indexes = np.random.choice(len(X_tst), size, replace=False)
X_new = X_tst.iloc[indexes]
y_new = MLR.predict(poly.fit_transform(X_new))

plot_predictions(X_new,y_new,descr = 'Linear Regression: ')

## 4.2. Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

d_tree = DecisionTreeRegressor()
d_tree.fit(X_trn,y_trn)

y_pr_tst = d_tree.predict(X_tst)

mae = mean_absolute_error(y_tst,y_pr_tst)
r2 = r2_score(y_tst,y_pr_tst)


model_mae['tree'] = mae
model_r2['tree'] = r2

print('MAE:{:7.2f},{:7.2f}% of mean'.format(mae,100*mae/y_pr_tst.mean()))
print('R2:{:6.3f}'.format(r2))

In [ ]:
# Getting a reduced sample to test
size = len(X_tst)
indexes = np.random.choice(len(X_tst), size, replace=False)
X_new = X_tst.iloc[indexes]
y_new = d_tree.predict(X_new)

plot_predictions(X_new,y_new,descr = 'Decision Tree: ')

## 4.3. Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rnd_frst = RandomForestRegressor()
rnd_frst.fit(X_trn,y_trn)

y_pr_tst = rnd_frst.predict(X_tst)

mae = mean_absolute_error(y_tst,y_pr_tst)
r2 = r2_score(y_tst,y_pr_tst)


model_mae['rnd_forest'] = mae
model_r2['rnd_forest'] = r2

print('MAE:{:7.2f},{:7.2f}% of mean'.format(mae,100*mae/y_pr_tst.mean()))
print('R2:{:6.3f}'.format(r2))

In [ ]:
# Getting a reduced sample to test
size = len(X_tst)
indexes = np.random.choice(len(X_tst), size, replace=False)
X_new = X_tst.iloc[indexes]
y_new = rnd_frst.predict(X_new)

plot_predictions(X_new,y_new,descr = 'Random Forest: ')

<a id="m-res"></a>
## 4.4. Results

In [ ]:
print('Minor Error: ',min(model_mae, key=model_mae.get),', (',min(list(model_mae.values())),')')
print('Best R2    : ',max(model_r2, key=model_r2.get),', (',max(list(model_r2.values())),')')

So, in most cases, the training procedure selects the **linear model** as the one that minimizes errors and maximizes R2 score.

<a id="real-model"></a>
# 5. Realistic Model

As stated before, in a real case, the user wants to predict the cost variables according to the characteristics of the property. Let's study how to make this approach here.

First of all, we have to drop all cost variables from dataset and split it into training and test.

In [ ]:
# Selecting features and target
#X = df1.drop('total',axis=1).copy()
X = df1.drop(['total','fire insurance','property tax','rent amount','hoa'],axis=1).copy()
y = df1['total'].copy()

# splitting
X_trn, X_tst, y_trn, y_tst = train_test_split(X, y, test_size=0.333)

# dictionary
model_mae = {}
model_r2 = {}

Now, let's test some regression methods and collect its metrics 

## Linear Regression

In [ ]:
# Create regression object
MLR = linear_model.LinearRegression()

poly = PolynomialFeatures(degree=1)
X_trn_pl = poly.fit_transform(X_trn)
X_tst_pl = poly.fit_transform(X_tst)
MLR.fit(X_trn_pl,y_trn)

y_pr_tst = MLR.predict(X_tst_pl)
mae = mean_absolute_error(y_tst,y_pr_tst)
r2 = r2_score(y_tst,y_pr_tst)

model_mae['linear'] = mae
model_r2['linear'] = r2

print('MAE:{:7.2f},{:7.2f}% of mean'.format(mae,100*mae/y_pr_tst.mean()))
print('R2:{:6.3f}'.format(r2))


# Getting a reduced sample to test
size = len(X_tst)
indexes = np.random.choice(len(X_tst), size, replace=False)
X_new = X_tst.iloc[indexes]
y_new = MLR.predict(poly.fit_transform(X_new))

plot_predictions(X_new,y_new,descr = 'REAL - Linear Regression: ',cols = ['area'])

## Quadratic Regression

In [ ]:
MLR = linear_model.LinearRegression()

poly = PolynomialFeatures(degree=2)
X_trn_pl = poly.fit_transform(X_trn)
X_tst_pl = poly.fit_transform(X_tst)
MLR.fit(X_trn_pl,y_trn)

y_pr_tst = MLR.predict(X_tst_pl)
mae = mean_absolute_error(y_tst,y_pr_tst)
r2 = r2_score(y_tst,y_pr_tst)

model_mae['quadratic'] = mae
model_r2['quadratic'] = r2

print('MAE:{:7.2f},{:7.2f}% of mean'.format(mae,100*mae/y_pr_tst.mean()))
print('R2:{:6.3f}'.format(r2))

# Getting a reduced sample to test
size = len(X_tst)
indexes = np.random.choice(len(X_tst), size, replace=False)
X_new = X_tst.iloc[indexes]
y_new = MLR.predict(poly.fit_transform(X_new))

plot_predictions(X_new,y_new,descr = 'REAL - Quadratic Regression: ',cols = ['area'])

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

d_tree = DecisionTreeRegressor()
d_tree.fit(X_trn,y_trn)

y_pr_tst = d_tree.predict(X_tst)

mae = mean_absolute_error(y_tst,y_pr_tst)
r2 = r2_score(y_tst,y_pr_tst)


model_mae['tree'] = mae
model_r2['tree'] = r2## Quadratic Regression

print('MAE:{:7.2f},{:7.2f}% of mean'.format(mae,100*mae/y_pr_tst.mean()))
print('R2:{:6.3f}'.format(r2))

# Getting a reduced sample to test
size = len(X_tst)
indexes = np.random.choice(len(X_tst), size, replace=False)
X_new = X_tst.iloc[indexes]
y_new = d_tree.predict(X_new)

plot_predictions(X_new,y_new,descr = 'REAL - Decision Tree: ',cols = ['area'])

## Random Forest 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rnd_frst = RandomForestRegressor()
rnd_frst.fit(X_trn,y_trn)

y_pr_tst = rnd_frst.predict(X_tst)

mae = mean_absolute_error(y_tst,y_pr_tst)
r2 = r2_score(y_tst,y_pr_tst)

model_mae['rnd_forest'] = mae
model_r2['rnd_forest'] = r2

print('MAE:{:7.2f},{:7.2f}% of mean'.format(mae,100*mae/y_pr_tst.mean()))
print('R2:{:6.3f}'.format(r2))

# Getting a reduced sample to test
size = len(X_tst)
indexes = np.random.choice(len(X_tst), size, replace=False)
X_new = X_tst.iloc[indexes]
y_new = rnd_frst.predict(X_new)

plot_predictions(X_new,y_new,descr = 'REAL - Random Forest: ',cols = ['area'])

In [ ]:
print('Minor Error: ',min(model_mae, key=model_mae.get),', (',min(list(model_mae.values())),')')
print('Best R2    : ',max(model_r2, key=model_r2.get),', (',max(list(model_r2.values())),')')

<a id="conc"></a>
# 6. Conclusion

Using the complete set of feature columns, we could fit a **Linear Model** which predicts the total rental value extremely well. The mean absolute error is something about **R\\$ 77,00** and the R2 **0.998**. However, this cannot be considered a real applicable model because, as we argued before, the cost variables used to define this model are correlated with the target. From a didactic perspective, it was possible to understand the influence of the number of features and its correlations on the quality of model and, in particular, it was possible to see that in certain situations, regression models can provide quite different results.

Following the same steps, I tried to build a realistic model using only the property's features, like  **city**, **area**, **rooms**, **bathroom**, **parking spaces**, **floor**, **animal** and **furniture** (that are the  features accessible to the user), to predict the **total**. In this case, the predictability of the models significantly reduces. The best regression approach for this considerations is the **Random Forest**, with mean error of **R\\$ 1848,75** and R2 score of **0.65**. Although these metrics are quite bad compared to the ideal case, it is possible to find advantages in using this model. The mean of **total** columns is **R\\$ 5816,84** with and the standard deviation of **R\\$ 4643,00**. So, as our model is predicting data with mean absolute error significantly less than the standard deviation, we can conclude that there exists an influence of this feature to the total rental cost. For a most efficient approach, however, it is necessary to consider a greater number of  features.